In [8]:
import os
import json
import torch
from tqdm import tqdm
from transformers import BartTokenizer
from transformers import BartForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [9]:
model = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=10)
model.load_state_dict(torch.load("model.pt"))


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [10]:
articles = []
tags = []

In [11]:
for cursor in tqdm(os.listdir("./normolized_aticles")):
    articles.append(cursor)

100%|███████████████████████████████████████████████████████████████████████| 24940/24940 [00:00<00:00, 4157298.38it/s]


In [12]:
for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for i in tmp:
        tags.append(i)

100%|██████████████████████████████████████████████████████████████████████████| 24940/24940 [00:03<00:00, 7828.01it/s]


In [13]:
element_counts = {}
for element in tqdm(tags):
    count = tags.count(element)
    element_counts[element] = count 

100%|█████████████████████████████████████████████████████████████████████████| 150929/150929 [04:35<00:00, 547.57it/s]


In [14]:
from operator import itemgetter

sorted_counts = sorted(element_counts.items(), key=itemgetter(1), reverse=True)

sorted_counts = sorted_counts[:10]
for element, count in sorted_counts:
    print(element, count)

проблема 676
python 607
apple 537
искусственный интеллект 507
программирование 481
игры 469
россия 430
информационная безопасность 407
машинное обучение 385
санкции 381


In [15]:
tags_top10 = []
for element, _ in sorted_counts:
        tags_top10.append(element)
        
print(tags_top10)
tags_top10_digit = []
index = 0
for tag in tags_top10:
    tmp = [tag, index]
    index += 1
    tags_top10_digit.append(tmp)

print(tags_top10_digit)

['проблема', 'python', 'apple', 'искусственный интеллект', 'программирование', 'игры', 'россия', 'информационная безопасность', 'машинное обучение', 'санкции']
[['проблема', 0], ['python', 1], ['apple', 2], ['искусственный интеллект', 3], ['программирование', 4], ['игры', 5], ['россия', 6], ['информационная безопасность', 7], ['машинное обучение', 8], ['санкции', 9]]


In [16]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

overall = 0
success = 0

for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for tag in tmp:
        if tag in tags_top10:
            test_texts = data["content"]
            encoded_test_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')
            test_input_ids = encoded_test_inputs['input_ids']
            test_attention_mask = encoded_test_inputs['attention_mask']
            
            with torch.no_grad():
                model.eval()
                test_outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask)

            test_predicted_labels = torch.argmax(test_outputs.logits, dim=1).tolist()

            for text, predicted_label in zip(test_texts, test_predicted_labels):
                print(f"Название текста: {data['article_name']}")
                print(f"Предсказанная метка: {tags_top10[predicted_label]}")
                print(f"Изначальный тег : {tag}\n")
            overall += 1
            if tag == tags_top10[predicted_label]:
                success += 1
overall /= 100
success /= overall
print(f"Success is: {success}")

  0%|                                                                                        | 0/24940 [00:00<?, ?it/s]

Название текста: Smart-lab: Nasdaq прекратит работу с резидентами из РФ с 29 апреля
Предсказанная метка: проблема
Изначальный тег : проблема



  0%|                                                                              | 5/24940 [00:05<8:13:40,  1.19s/it]

Название текста: Smart-lab: Nasdaq прекратит работу с резидентами из РФ с 29 апреля
Предсказанная метка: проблема
Изначальный тег : санкции

Название текста: Друг или враг? Стоит ли бояться искусственного интеллекта?
Предсказанная метка: искусственный интеллект
Изначальный тег : искусственный интеллект



  0%|                                                                             | 15/24940 [00:13<5:57:07,  1.16it/s]

...

Название текста: Nintendo пояснила, что завершает работу в РФ и расстаётся с российскими сотрудниками
Предсказанная метка: санкции
Изначальный тег : санкции



100%|█████████████████████████████████████████████████████████████████████████▉| 24911/24940 [5:30:17<00:14,  1.99it/s]

Название текста: Механизм автопилота мозга управляет сознанием
Предсказанная метка: машинное обучение
Изначальный тег : машинное обучение



100%|██████████████████████████████████████████████████████████████████████████| 24940/24940 [5:30:21<00:00,  1.26it/s]

Название текста: Куда сходить: главные ИТ-события второй половины апреля
Предсказанная метка: информационная безопасность
Изначальный тег : информационная безопасность

Success is: 86.27049180327869
